# PSET 2: Matrix Inverse

...

Try not to add additional cells nor rearrange any that exist now as it will mess with our autograder (but feel free to open another Colab/notebook on the side).

### Make sure to submit your final notebook with all of your solutions to Gradescope!
[Direct Gradescope Link](https://www.gradescope.com/courses/820552)

### Before Starting

This problem set uses NVCC (NVIDIA CUDA Compiler) within Google Colab to compile and run CUDA code. Google Colab provides a convenient (and free!) environment with GPU support for executing these programs.  At the top right corner, under the arrow dropdown next to the `Connect` button, select "Change runtime type" and choose `T4 GPU` for this problem set.  Note, across your account, you can only have one runtime connected to a GPU at a given time.

We're in an interactive _Python_ notebook.  So, of course, running C++ code is not going to be as easy as it is to run Python code.  That said, we'll once again use helpers to make this cleaner.

Our plugin `%%gpurun` saves, compiles, and runs your CUDA code in the cell.

In [ ]:
# make sure CUDA is installed
!nvcc --version

# make sure you have a GPU runtime (if this fails go to runtime -> change runtime type)
!nvidia-smi

# Install some magic to run and save .cu C++ CUDA programs
!curl -o ./gpu_runner.py https://raw.githubusercontent.com/COMS-BC3159-F24/helpers/main/gpu_runner.py
%load_ext gpu_runner

# to learn about how to do more fancy things with CUDA using this API see:
# https://nvcc4jupyter.readthedocs.io/en/latest/index.html

## Problem 1: Inverse!

In the previous problem sets, you've explored how we can compute matrix multiplication via various methods and platforms (naïvely in Python, using libraries, C++, memory-aware methods, and on a GPU).  That is one piece necessary for programatically solving a linear system. *Now*, let's write a kernel to compute a **matrix inverse** which we can then combine with our prior matrix multiplication to solve a linear system.


As always, we have provided some starter code for you, so your job is to fill in the missing pieces.  We've structured the code for you, so let's describe the pieces:


### `matrix_inverse_inner` (5 points) 
This device function will do all of the heavy lifting and actually implement the matrix inverse function. It takes in the input matrix and outputs the inverse. You should assume these pointers are already in shared memory. There is also an additional input of additional shared temporary memory which you may or may not need to use (and you will get to specify how big it is in later functions). You can implement this in any way, but the simplest solution is to use the Gauss-Jordan elimination method. You can see some graphical examples of "elementary row operations" [at this link](https://www.mathsisfun.com/algebra/matrix-inverse-row-operations-gauss-jordan.html). 


You can simply walk through the rows in order (do not swap any rows) and for each row divide it to ensure that it is leading with a 1 and then subtract down along the rest of the rows to ensure they are all leading with a 0. If you repeat this and move down and to the right through the matrix, you will end up with the identity in place of the original matrix and the inverse to the right! 

You can assume the following:
- the input matrix does not have a zero in the top left corner
- the input is of a relatively small size (less than 30x30)
- the input is a square matrix. 

The function input `matrix_dim` is the number of rows of the square input matrix.

**Note:** there are a number of ways to actually implement matrix inverse. Start with whatever is simplest, then feel free to optimize!

### `matrix_inverse_kernel` (2 points)
This kernel will call the `matrix_inverse_inner` function and should do the following:
1. move the device matrix into shared memory, 
2. make sure the computation occurs in shared memory, and
3. return the value to device memory. 

You will note that we specified dynamic shared memory. Please use that and make sure to allocate enough of it in the host function to support however much your device function needs! As with `matrix_inverse_inner`, the function input `matrix_dim` is the number of rows of the input matrix.

###  `matrix_inverse` (1 point)
This host function will call the `matrix_inverse_kernel` and should do the following:
1. move the input host matrix into device memory, 
2. launch the kernel (with dynamic shared memory), and 
3. return the solution to host memory. 

Again, the function input `matrix_dim` is the number of rows of the input matrix.

**Autograder**: the Gradescope autograder will return failure since this problem set requires a GPU.  To check your solution, here's a Python implementation for reference.  The result of your CUDA code should exactly match the Python output below.

In [ ]:
import numpy as np
np.set_printoptions(precision=6, floatmode='fixed')

def printer(arr):
    for row in arr:
        print(" ".join(f"{x:.6f}" for x in row))
    print()

### FIRST TEST
test1 = 2*np.eye(5)
printer(test1)
printer(np.linalg.inv(test1))

### SECOND TEST
test2 = 7*np.tri(5)
printer(test2)
printer(np.linalg.inv(test2))

In [ ]:
%%gpurun -n cuda_mat_inv.cu
#include <stdio.h>

// GPU Device Function
// - actually solve the matrix inverse!
__device__
void matrix_inverse_inner(float *s_input, float *s_output, float *s_temp, const int matrix_dim){
  // Set up the matrix with identity next to it
  ...
  ...
  ...
  ...


  // Do Guassian elimination walking down the matrix (assuming no leading 0s).
  // We therefore use the columns in order as the pivot column for each pivot we need to rescale
  // that row so that the pivot value is 1 THEN for all other row values we need to add a multiple
  // of the NEW pivot row value such that we transorm the other row pivot column value to 0.
  // See https://www.mathsisfun.com/algebra/matrix-inverse-row-operations-gauss-jordan.html
  //
  // Note if you would prefer to use another method that is fine but/and this is the method
  // we have a solution for and are prepared to help you with!

  for (unsigned pivRC = 0; pivRC < matrix_dim; pivRC++){
      ...
      ...
      ...
      ...
      ...
      ...
      ...
      ...
      ...
  }

  // Make sure to write the result to the output
  ...
  ...
  ...
  ...
}


// GPU kernel
// - Set up shared memory, run the _inner, clean up shared memory
__global__
void matrix_inverse_kernel(float *d_input, float *d_output, const int matrix_dim){
  // get shared pointers
  extern __shared__ float s_dynShared[];
  ...
  ...
  ...
  ...

  // copy the d_input data into shared memory
  ...
  ...
  ...
  ...

  // run the code
  ...
  ...

  // copy the memory back out to d_output
  ...
  ...
  ...
  ...
}


// Host function
// - set up GPU memory, run the kernel, clean up GPU memory
__host__
void matrix_inverse(float *h_input, float *h_output, const int matrix_dim){
  // transfer memory to the device
  ...
  ...
  ...
  ...

  // run the kernel
  ...
  ...
  ...
  ...

  // transfer data back to the host and clean up
  ...
  ...
  ...
  ...
}


// ************************************************
//  Main and Utility Functions
// ------------------------------------------------
//  - You do not need to modify these.
//  - They should match the Python
//    reference above if your code works!
// ************************************************

__host__
void printMat(float *mat, const int matrix_dim){
    // loop through row by row and print
    for (int r = 0; r < matrix_dim; r++){
        for (int c = 0; c < matrix_dim; c++){
            printf("%f ",mat[r + c*matrix_dim]);
        }
        // Newline for new row
        printf("\n");
    }
    // Newline for end of print
    printf("\n");
}

__host__
void runTest(float *h_input, float *h_output, const int matrix_dim){
  // print the input matrix
  printMat(h_input,matrix_dim);

  // run the main function
  matrix_inverse(h_input,h_output,matrix_dim);

  // print the final result
  printMat(h_output,matrix_dim);
}

__host__
int main() {

  // initialize the first test matrix
  const int matrix_dim = 5;
  const int matrix_dim_sq = matrix_dim*matrix_dim;
  float *h_input = (float *)malloc(matrix_dim_sq*sizeof(float));
  float *h_output = (float *)malloc(matrix_dim_sq*sizeof(float));
  for (int c = 0; c < matrix_dim; c++){
      for (int r = 0; r < matrix_dim; r++){
          h_input[r + c*matrix_dim] = (r == c) ? 2 : 0;
      }
  }
  // run the test
  runTest(h_input,h_output,matrix_dim);

  // now do the second test
  for (int c = 0; c < matrix_dim; c++){
      for (int r = 0; r < matrix_dim; r++){
          h_input[r + c*matrix_dim] = r >= c ? 7 : 0;
      }
  }
  runTest(h_input,h_output,matrix_dim);

  return 0;
}